In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import config
import cx_Oracle
import datetime

In [12]:
import functions
from importlib import reload
reload(functions)
from functions import *

## Catégorie

In [3]:
# définition des variables
nom_flux='CATEGORIE'
cle = 'CD_CATEGORIE'

# TRAITEMENT #

# import du fichier
df, nom_fichier, nom_flux = import_fichier(nom_flux=nom_flux)

# traitements df
df = ajout_col_num_ligne(df)
df, df_na = traitement_valeurs_nulles(df)
df, df_avec_doublons = traitement_doublons(df)
df, colonnes_fichier = traitement_pre_import(df, nom_fichier, nom_flux)

# traitements rejets
rejet_valeurs_nulles = creation_rejet_valeurs_nulles(df_na, nom_fichier, nom_flux)
rejet_doublons = creation_rejet_doublons(df_avec_doublons, nom_fichier, nom_flux)
rejets = rassemblement_rejet(rejet_valeurs_nulles, rejet_doublons)


# BASE DE DONNEES #

# ouverture de la connexion et du curseur
conn = open_connection()
cur = open_cursor(conn)

# insertion des rejets s'il y en a
if rejets.shape[0] > 0 : 
    insertion_rejets(rejets, cur, conn)
else :
    print(f"Pas de rejets dans le fichier {nom_fichier}")


# INSERTION NOUVELLES VALEURS
    
# recherche de nouvelles valeurs dans le fichier
resultat_requete_selection_table = selection_table(colonnes_fichier, cur, nom_flux)
lignes_a_inserer = detection_nouvelles_valeurs(colonnes_fichier, cle, resultat_requete_selection_table, df, nom_flux)

# insertion des valeurs trouvées
if lignes_a_inserer.shape[0]>0 :
    insertion_nouvelles_valeurs(lignes_a_inserer, conn, cur, nom_flux)


# MODIFICATION ANCIENNES VALEURS, MISES OBSOLETES
    
# recherche de valeurs modifiées
jointure_fichier_table = detection_valeurs_modifiees_jointure(colonnes_fichier, cle, resultat_requete_selection_table, df)
# mask à modifier, spécifique à la table
mask = (jointure_fichier_table['LB_CATEGORIE_FICHIER'] != jointure_fichier_table['LB_CATEGORIE_TABLE'])
lignes_a_modifier = detection_valeurs_modifiees_mask(jointure_fichier_table, mask, nom_flux, cle, df)

if lignes_a_modifier.shape[0] > 0 :
    modification_valeurs(lignes_a_modifier, conn, cur, nom_flux, cle)
    insertion_nouvelles_valeurs_modifiees(lignes_a_modifier, conn, cur, nom_flux)

# fermeture du curseur et de la connexion
close_cursor(cur)
close_connection(conn)

CATEGORIE
----------------------------------------------------------------------------------------------------
Fichier ICOM_20240117_CATEGORIE.csv trouvé
----------------------------------------------------------------------------------------------------
Fichier ICOM_20240117_CATEGORIE.csv importé
----------------------------------------------------------------------------------------------------
Nombre de lignes qui contiennent des valeurs nulles : 0
Nombre de lignes qui contiennent des doublons : 0
----------------------------------------------------------------------------------------------------
Nombre de lignes rejetées : 0
----------------------------------------------------------------------------------------------------
Connexion établie.
Curseur ouvert.
Pas de rejets dans le fichier ICOM_20240117_CATEGORIE.csv
Requête de sélection de la table ODS_CATEGORIE réussie, 0 lignes trouvées.
3 nouvelles lignes à insérer dans la table ODS_CATEGORIE.
3 nouvelles lignes insérées dans la 

In [4]:
conn = open_connection()
cur = open_cursor(conn)

print(selection_table_entiere(df.columns, cur, nom_flux))

close_cursor(cur)
close_connection(conn)

Connexion établie.
Curseur ouvert.
  CD_CATEGORIE           LB_CATEGORIE               lb_nom_fichier  \
0         C001            Technologie  ICOM_20240117_CATEGORIE.csv   
1         C002               Mobilier  ICOM_20240117_CATEGORIE.csv   
2         C003  Fournitures de bureau  ICOM_20240117_CATEGORIE.csv   

         dt_insertion  bl_ligne_active lb_job_name  
0 2024-03-31 16:06:15                1   CATEGORIE  
1 2024-03-31 16:06:15                1   CATEGORIE  
2 2024-03-31 16:06:15                1   CATEGORIE  
Curseur fermée.
Connexion fermée.


In [5]:
conn = open_connection()
cur = open_cursor(conn)

suppression_donnees_table(cur, nom_flux='CATEGORIE', conn=conn)
suppression_donnees_table(cur, nom_flux='TYPE_CLIENT', conn=conn)
suppression_donnees_table(cur, nom_flux='REJET', conn=conn)

close_cursor(cur)
close_connection(conn)

Connexion établie.
Curseur ouvert.
Données de la table ODS_CATEGORIE supprimées.
Données de la table ODS_TYPE_CLIENT supprimées.
Données de la table ODS_REJET supprimées.
Curseur fermée.
Connexion fermée.


## Création table temporaire

In [21]:
df

,CD_CATEGORIE,LB_CATEGORIE,lb_nom_fichier,dt_insertion,bl_ligne_active,lb_job_name
0,C001,Technologie,ICOM_20240117_CATEGORIE.csv,2024-03-30 16:55:20.946530,1,CATEGORIE
1,C002,Mobilier,ICOM_20240117_CATEGORIE.csv,2024-03-30 16:55:20.946530,1,CATEGORIE
2,C003,Fournitures de bureau,ICOM_20240117_CATEGORIE.csv,2024-03-30 16:55:20.946530,1,CATEGORIE


In [36]:
def creation_table_temporaire_fichier(conn, cur, nom_flux, df):
    ''''Création d'une table temporaire pour intégrer le fichier.
    Intégration des données dans la table'''

    try : 
        cur.execute(f"drop table {nom_flux}_FICHIER")
        print(f"Suppression de la table {nom_flux}_FICHIER réussie.")
    except cx_Oracle.DatabaseError as error:
        print(f"Tentative de suppression de la table {nom_flux}_FICHIER échoué : {error}")


    requete_creation_table_temporaire = f'''
    create global temporary table {nom_flux}_FICHIER
    on commit preserve rows
    as 
    select * from ods_{nom_flux}
    where 1=0
    '''
    # création table temporaire
    try :
        cur.execute(requete_creation_table_temporaire)
        print(f"Table temporaire {nom_flux}_FICHIER créée.")
    except cx_Oracle.DatabaseError as error:
        print(f"Erreur lors de la création de la table temporaire {nom_flux}_FICHIER.")

    # insertion des données
    insertion_fichier_table_temporaire(df, conn, cur, nom_flux)

In [17]:
conn = open_connection()
cur = open_cursor(conn)
creation_table_temporaire_fichier(conn, cur, nom_flux, df)

Connexion établie.
Curseur ouvert.
Suppression de la table CATEGORIE_FICHIER réussie.
Table temporaire CATEGORIE_FICHIER créée.
3 lignes insérées dans la table CATEGORIE_FICHIER


In [18]:
selection_table_entiere(colonnes_fichier=df.columns, cur=cur, nom_table='CATEGORIE_FICHIER')

,CD_CATEGORIE,LB_CATEGORIE,lb_nom_fichier,dt_insertion,bl_ligne_active,lb_job_name
0,C001,Technologie,ICOM_20240117_CATEGORIE.csv,2024-03-31 16:06:15,1,CATEGORIE
1,C002,Mobilier,ICOM_20240117_CATEGORIE.csv,2024-03-31 16:06:15,1,CATEGORIE
2,C003,Fournitures de bureau,ICOM_20240117_CATEGORIE.csv,2024-03-31 16:06:15,1,CATEGORIE


In [29]:
for col in colonnes_fichier[:-1]:
    print(f"f.{col} = t.{col} and")

f.CD_CATEGORIE = t.CD_CATEGORIE and


In [30]:
cle

'CD_CATEGORIE'

In [ ]:


requete_nouvelles_valeurs = f'''
select * from {nom_flux}_FICHIER f
left join ODS_{nom_flux} t on f.
'''
cur.execute

## Type client

In [107]:
# définition des variables
nom_flux='TYPE_CLIENT'
cle = 'CD_TYPE_CLIENT'

# TRAITEMENT #

# import du fichier
df, nom_fichier, nom_flux = import_fichier(nom_flux=nom_flux)

# traitements df
df = ajout_col_num_ligne(df)
df, df_na = traitement_valeurs_nulles(df)
df, df_avec_doublons = traitement_doublons(df)
df, colonnes_fichier = traitement_pre_import(df, nom_fichier, nom_flux)

# traitements rejets
rejet_valeurs_nulles = creation_rejet_valeurs_nulles(df_na, nom_fichier, nom_flux)
rejet_doublons = creation_rejet_doublons(df_avec_doublons, nom_fichier, nom_flux)
rejets = rassemblement_rejet(rejet_valeurs_nulles, rejet_doublons)


# BASE DE DONNEES #

# ouverture de la connexion et du curseur
conn = open_connection()
cur = open_cursor(conn)

# insertion des rejets s'il y en a
if rejets.shape[0] > 0 : 
    insertion_rejets(rejets, cur, conn)
else :
    print(f"Pas de rejets dans le fichier {nom_fichier}")


# INSERTION NOUVELLES VALEURS
    
# recherche de nouvelles valeurs dans le fichier
resultat_requete_selection_table = selection_table(colonnes_fichier, cur, nom_flux)
lignes_a_inserer = detection_nouvelles_valeurs(colonnes_fichier, cle, resultat_requete_selection_table, df, nom_flux)

# insertion des valeurs trouvées
if lignes_a_inserer.shape[0]>0 :
    insertion_nouvelles_valeurs(lignes_a_inserer, conn, cur, nom_flux)


# MODIFICATION ANCIENNES VALEURS, MISES OBSOLETES
    
# recherche de valeurs modifiées
jointure_fichier_table = detection_valeurs_modifiees_jointure(colonnes_fichier, cle, resultat_requete_selection_table, df)
# mask à modifier, spécifique à la table
mask = (jointure_fichier_table['LB_TYPE_CLIENT_FICHIER'] != jointure_fichier_table['LB_TYPE_CLIENT_TABLE'])
lignes_a_modifier = detection_valeurs_modifiees_mask(jointure_fichier_table, mask, nom_flux, cle, df)

if lignes_a_modifier.shape[0] > 0 :
    modification_valeurs(lignes_a_modifier, conn, cur, nom_flux, cle)
    insertion_nouvelles_valeurs_modifiees(lignes_a_modifier, conn, cur, nom_flux)

# fermeture du curseur et de la connexion
close_cursor(cur)
close_connection(conn)

TYPE_CLIENT
----------------------------------------------------------------------------------------------------
Fichier ICOM_20240117_TYPE_CLIENT.csv trouvé
----------------------------------------------------------------------------------------------------
Fichier ICOM_20240117_TYPE_CLIENT.csv importé
----------------------------------------------------------------------------------------------------
Nombre de lignes qui contiennent des valeurs nulles : 0
Nombre de lignes qui contiennent des doublons : 0
----------------------------------------------------------------------------------------------------
Nombre de lignes rejetées : 0
----------------------------------------------------------------------------------------------------
Connexion établie.
Curseur ouvert.
Pas de rejets dans le fichier ICOM_20240117_TYPE_CLIENT.csv
Requête de sélection de la table ODS_TYPE_CLIENT réussie, 3 lignes trouvées.
0 nouvelles lignes à insérer dans la table ODS_TYPE_CLIENT.
0 lignes à modifier dans

## Factorisation